<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumToyAlgorithm_QuantumTeleportation_00_mkbahk_20250318.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-aer==0.15.1
%pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.2 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=863dbded201c81832b71fa198f78f13252a97d34a45a5b6af280ff643db11612
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.8 MB/s eta 0:00:00
  Cloning https://github.com/qis

In [ ]:
%pip show qiskit
%pip show qiskit-aer

Name: qiskit
Version: 1.2.4
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer, qiskit-textbook
Name: qiskit-aer
Version: 0.15.1
Summary: Aer - High performance simulators for Qiskit
Home-page: https://github.com/Qiskit/qiskit-aer
Author: AER Development Team
Author-email: qiskit@us.ibm.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, psutil, qiskit, scipy
Required-by: 


In [4]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit_textbook.tools import array_to_latex
from IPython.display import display, Math
import numpy as np
from numpy import pi

In [10]:
# 양자 텔레포테이션 회로 생성 함수
def create_quantum_teleportation_circuit(initial_state=None):
    """
    양자 텔레포테이션 회로를 생성합니다.

    Args:
        initial_state (list): 텔레포트할 상태의 파라미터 [theta, phi] (None인 경우 랜덤 상태 생성)

    Returns:
        qc: 완성된 양자 텔레포테이션 회로
    """
    # 양자 레지스터와 고전 레지스터 생성
    qr = QuantumRegister(3, name='q')  # 0: Alice의 전송할 큐비트, 1: Alice의 얽힘 큐비트, 2: Bob의 큐비트
    crz = ClassicalRegister(1, name='crz')  # Alice의 Z-측정 결과
    crx = ClassicalRegister(1, name='crx')  # Alice의 X-측정 결과
    qc = QuantumCircuit(qr, crz, crx)

    # 1. 전송할 상태 준비 (초기 상태가 제공되지 않은 경우 랜덤 상태 생성)
    if initial_state is None:
        theta = np.random.random() * np.pi
        phi = np.random.random() * 2 * np.pi
    else:
        theta, phi = initial_state
    ###if

    qc.ry(theta, qr[0])
    qc.rz(phi, qr[0])
    qc.barrier()

    # 2. Bell 상태 생성 (Alice와 Bob 사이의 얽힘)
    qc.h(qr[1])
    qc.cx(qr[1], qr[2])
    qc.barrier()

    # 3. Alice의 Bell 측정 준비
    qc.cx(qr[0], qr[1])
    qc.h(qr[0])
    qc.barrier()

    # 4. Alice의 측정 및 결과 저장
    qc.measure(qr[0], crx)
    qc.measure(qr[1], crz)
    qc.barrier()

    # 5. Bob의 보정 연산 (측정 결과에 따른 조건부 연산)
    with qc.if_test((crx, 1)):
        qc.z(qr[2])
    ###with

    with qc.if_test((crz, 1)):
        qc.x(qr[2])
    ###with

    return qc, [theta, phi]
###def

In [15]:
# 양자 텔레포테이션 시뮬레이션 실행 함수
def run_quantum_teleportation(circuit, num_shots=1024):
    """
    양자 텔레포테이션 회로를 시뮬레이션합니다.

    Args:
        circuit: 양자 텔레포테이션 회로
        num_shots: 시뮬레이션 횟수

    Returns:
        result: 시뮬레이션 결과
    """
    simulator = Aer.get_backend('aer_simulator')
    compiled_circuit = transpile(circuit, simulator)
    result = simulator.run(compiled_circuit, shots=num_shots).result()
    return result
###def

In [16]:
# 검증 회로 생성 함수
def create_verification_circuit(initial_state):
    """
    텔레포테이션 결과를 검증하기 위한 회로를 생성합니다.

    Args:
        initial_state (list): 텔레포트된 상태의 파라미터 [theta, phi]

    Returns:
        qc: 검증을 위한 양자 회로
    """
    theta, phi = initial_state

    # 원본 상태를 담을 큐비트
    qr_original = QuantumRegister(1, name='original')
    # 텔레포트된 상태를 담을 큐비트
    qr_teleported = QuantumRegister(1, name='teleported')
    # 측정 결과를 담을 레지스터
    cr_original = ClassicalRegister(1, name='m_orig')
    cr_teleported = ClassicalRegister(1, name='m_tele')

    qc = QuantumCircuit(qr_original, qr_teleported, cr_original, cr_teleported)

    # 원본 상태 준비
    qc.ry(theta, qr_original)
    qc.rz(phi, qr_original)

    # 텔레포트된 상태 준비 (이미 텔레포트되었다고 가정)
    qc.ry(theta, qr_teleported)
    qc.rz(phi, qr_teleported)

    # Z-기저 측정을 위한 준비
    qc.h(qr_original)
    qc.h(qr_teleported)

    # 측정
    qc.measure(qr_original, cr_original)
    qc.measure(qr_teleported, cr_teleported)

    return qc
###def

In [17]:
def main():
    # 양자 텔레포테이션 회로 생성 (|ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩)
    initial_state = [np.pi/4, np.pi/2]  # 텔레포트할 상태 (임의로 설정)
    teleportation_circuit, state_params = create_quantum_teleportation_circuit(initial_state)

    # 회로 출력
    print("양자 텔레포테이션 회로:")
    print(teleportation_circuit)

    # 회로 시뮬레이션
    result = run_quantum_teleportation(teleportation_circuit, num_shots=1024)
    counts = result.get_counts()

    print("\n측정 결과:")
    print(counts)

    # 검증 회로 실행 (원본 상태와 텔레포트된 상태가 같은지 확인)
    verification_circuit = create_verification_circuit(state_params)
    sim = Aer.get_backend('aer_simulator')
    compiled_verification = transpile(verification_circuit, sim)
    verification_result = sim.run(compiled_verification, shots=1024).result()
    verification_counts = verification_result.get_counts()

    print("\n검증 결과 (원본과 텔레포트된 상태 비교):")
    print(verification_counts)

    print("\n텔레포트된 상태 파라미터:")
    print(f"theta = {state_params[0]}, phi = {state_params[1]}")
###def

In [18]:
# 메인 예제 코드
if __name__ == "__main__":
    main()

양자 텔레포테이션 회로:
       ┌─────────┐┌─────────┐ ░            ░      ┌───┐ ░ ┌─┐    ░         »
  q_0: ┤ Ry(π/4) ├┤ Rz(π/2) ├─░────────────░───■──┤ H ├─░─┤M├────░─────────»
       └─────────┘└─────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░         »
  q_1: ───────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░─────────»
                              ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ ┌────── »
  q_2: ───────────────────────░──────┤ X ├─░────────────░──╫──╫──░─┤ If-0  »
                              ░      └───┘ ░            ░  ║  ║  ░ └──╥─── »
crz: 1/════════════════════════════════════════════════════╬══╩═══════╬════»
                                                           ║  0    ┌──╨──┐ »
crx: 1/════════════════════════════════════════════════════╩═══════╡ 0x1 ╞═»
                                                           0       └─────┘ »
«                                             
«  q_0: ──────────────────────────────────────
«                                            